# LDA Topic Models with Varying Parameters

*Apply LDA with different values for alpha and beta parameters. Export the models for visualisation.*

In [42]:
import json

from gensim.models.ldamodel import LdaModel
from gensim import corpora

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

from collections import Counter
from itertools import chain

## Data Preprocessing

#### Load the data

In [7]:
def prepare_documents_facebook(filename):
    'Return a list of documents ready for topic modelling.'
    with open(filename) as f:
        posts = json.load(f)
        return [post['message'] for post in posts if 'message' in post]

In [6]:
data = prepare_documents_facebook('../data/facebook.json')

#### Clean the data

In [94]:
def preprocess_documents(docs):
    'Return a bag-of-word representation of the documents after cleaning (stopwords/punctuation removal, stemming).'
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation + '“”’—') 
    lemma = WordNetLemmatizer()
    return [clean(doc, stop, exclude, lemma) for doc in docs]

def clean(doc, stop, exclude, lemma):
    punc_free_doc = ''.join(c for c in doc if c not in exclude)
    stop_free_words = [w for w in punc_free_doc.lower().split() if w not in stop]    
    normalized_words = [lemma.lemmatize(w) for w in stop_free_words if len(w) >= 3]
    return normalized_words

In [95]:
docs = preprocess_documents(data)
for i in range(0, 10):
    print(data[i], docs[i])
    print()

The Syrian military declared today that the U.S.-Russia brokered cease-fire is over, blaming rebel groups for violating the agreement. ['syrian', 'military', 'declared', 'today', 'usrussia', 'brokered', 'ceasefire', 'blaming', 'rebel', 'group', 'violating', 'agreement']

Rose Pak, an influential community activist who turned San Francisco's Asian-American population into a political power in the city, passes away at 68. ['rose', 'pak', 'influential', 'community', 'activist', 'turned', 'san', 'franciscos', 'asianamerican', 'population', 'political', 'power', 'city', 'pass', 'away']

Warplanes target the besieged Syrian city of Aleppo for the first time since the ceasefire went into effect last week. ['warplane', 'target', 'besieged', 'syrian', 'city', 'aleppo', 'first', 'time', 'since', 'ceasefire', 'went', 'effect', 'last', 'week']

Using Skittles to make a point about “our Syrian refugee problem” didn’t go over too well with Mars, Incorporated—but the candy maker’s rebuke of Donald Tr

#### Look at word frequency

In [96]:
word_dict = Counter(chain.from_iterable(docs))
word_counts = sorted(word_dict.items(), key=lambda x: -x[1])
word_counts[:10]

[('trump', 204),
 ('donald', 165),
 ('clinton', 110),
 ('hillary', 95),
 ('say', 80),
 ('president', 74),
 ('debate', 63),
 ('presidential', 51),
 ('obama', 47),
 ('first', 40)]

## Topic Modelling with LDA

#### Find topics of the whole corpus

In [190]:
def build_lda(corpus, num_topics=10, passes=10, alpha='symmetric', eta=None):
    'Return an LDA model from the given doc-term matrix .'
    return LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=passes, alpha=alpha, eta=eta, random_state=0)

def get_model_topics(lda):
    return [[(lda.id2word[t], '{:.3f}'.format(p)) for t, p in lda.get_topic_terms(i)] for i in range(lda.num_topics)]

In [191]:
dictionary = corpora.Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]    
lda = build_lda(corpus, num_topics=10)
get_model_topics(lda)

[[('trump', '0.042'),
  ('donald', '0.036'),
  ('clinton', '0.025'),
  ('hillary', '0.021'),
  ('presidential', '0.014'),
  ('debate', '0.013'),
  ('republican', '0.008'),
  ('campaign', '0.007'),
  ('first', '0.007'),
  ('white', '0.006')],
 [('say', '0.015'),
  ('trump', '0.013'),
  ('would', '0.012'),
  ('donald', '0.011'),
  ('republican', '0.007'),
  ('voter', '0.007'),
  ('excited', '0.006'),
  ('think', '0.006'),
  ('first', '0.006'),
  ('house', '0.006')],
 [('president', '0.016'),
  ('trump', '0.011'),
  ('saying', '0.008'),
  ('donald', '0.008'),
  ('black', '0.008'),
  ('obama', '0.007'),
  ('american', '0.007'),
  ('hillary', '0.007'),
  ('clinton', '0.007'),
  ('political', '0.006')],
 [('trump', '0.019'),
  ('donald', '0.018'),
  ('say', '0.014'),
  ('new', '0.010'),
  ('great', '0.008'),
  ('president', '0.007'),
  ('know', '0.007'),
  ('campaign', '0.006'),
  ('back', '0.006'),
  ('debate', '0.006')],
 [('president', '0.016'),
  ('obama', '0.016'),
  ('year', '0.008'),


#### Get topics associated with documents

In [194]:
def get_topics_for_documents(lda, corpus):
    'Return topic probabilities for each document in the given corpus. Only topics with probability higher than 0.01.'
    return [get_topics_for_one_document(lda, doc) for doc in corpus]

def get_topics_for_one_document(lda, doc):
    return [float('{:.3f}'.format(p)) for (t, p) in lda.get_document_topics(doc, minimum_probability=0.01)]

In [196]:
output_topics = get_topics_for_documents(lda, corpus)
output_topics[:10]

[[0.931],
 [0.944],
 [0.94],
 [0.959],
 [0.975],
 [0.925],
 [0.936],
 [0.95],
 [0.961],
 [0.94]]

#### Find terms associated with topics 

In [198]:
def get_terms_for_topics(lda):
    'Return term probabilities for each topic. Only top 10 terms.'
    return [get_terms_for_one_topic(lda, i) for i in range(lda.num_topics)]

def get_terms_for_one_topic(lda, topic_id):
    return [float('{:.3f}'.format(p)) for t, p in lda.get_topic_terms(topic_id)]

In [199]:
output_terms = get_terms_for_topics(lda)
output_terms

[[0.042, 0.036, 0.025, 0.021, 0.014, 0.013, 0.008, 0.007, 0.007, 0.006],
 [0.015, 0.013, 0.012, 0.011, 0.007, 0.007, 0.006, 0.006, 0.006, 0.006],
 [0.016, 0.011, 0.008, 0.008, 0.008, 0.007, 0.007, 0.007, 0.007, 0.006],
 [0.019, 0.018, 0.014, 0.01, 0.008, 0.007, 0.007, 0.006, 0.006, 0.006],
 [0.016, 0.016, 0.008, 0.008, 0.007, 0.007, 0.006, 0.006, 0.006, 0.005],
 [0.022, 0.017, 0.013, 0.01, 0.01, 0.01, 0.01, 0.007, 0.007, 0.006],
 [0.018, 0.013, 0.013, 0.01, 0.009, 0.008, 0.008, 0.006, 0.006, 0.006],
 [0.013, 0.009, 0.008, 0.008, 0.007, 0.006, 0.006, 0.005, 0.005, 0.005],
 [0.021, 0.014, 0.014, 0.014, 0.012, 0.012, 0.009, 0.007, 0.007, 0.007],
 [0.029, 0.028, 0.028, 0.023, 0.013, 0.012, 0.011, 0.011, 0.008, 0.007]]

## Export Model Data

In [200]:
def export_model_data(lda, corpus):
    'Return a dictionary detailing model parameters and probability matrices.'
    return {
        'alpha': float(lda.alpha[0]), # assume the same value for all elements
        'beta': float(lda.eta[0]), # assume the same value for all elements
        'num_topics': lda.num_topics,
        'doc_topics': get_topics_for_documents(lda, corpus),
        'topic_terms': get_terms_for_topics(lda)
    }

In [201]:
def save_file(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f)

#### Different alpha values

In [202]:
def export_data_for_alphas(corpus, alphas, filename):
    data = [export_model_data(build_lda(corpus, alpha=alpha), corpus) for alpha in alphas]
    save_file(data, filename)

In [ ]:
export_data_for_alphas(corpus, [0.01, 0.03, 0.1, 0.3, 1, 3, 10], '../data/facebook-alphas.json')